In [89]:
import random
import os

In [90]:
def adicionar_nova_sequencia_ao_dicionario(dicionario, nova_sequencia):
    proximo_codigo = len(dicionario)
    dicionario[nova_sequencia] = proximo_codigo

In [91]:
def criar_dicionario_inicial():
    dicionario = {}
    for i in range(256):
        dicionario[(i.to_bytes(1,byteorder='big',signed=False))] = i
    return dicionario

In [92]:
def compressao_LZW(nome_arquivo, tamanho_maximo):
    dicionario = criar_dicionario_inicial()
    tam_max_dic = 2 ** tamanho_maximo
    buffer = b''
    codigos = []

    with open(nome_arquivo, 'rb') as arquivo:
        while True:
            byte = arquivo.read(1)

            if not byte:
                break
            
            nova_sequencia = buffer + byte

            if nova_sequencia in dicionario:
                buffer = nova_sequencia
            else:
                if(len(dicionario) < tam_max_dic): # verifica tamanho maximo do dicionario
                    adicionar_nova_sequencia_ao_dicionario(dicionario, nova_sequencia)
                codigos.append(dicionario[buffer])
                buffer = byte

    if buffer:
        codigos.append(dicionario[buffer])
    #print(dicionario)
    return dicionario

In [93]:
def rec_pad(nome_arquivo, dicionario):
    #tam_max_dic = 2 ** tamanho_maximo
    buffer = b''
    codigos = []

    with open(nome_arquivo, 'rb') as arquivo:
        cabe = arquivo.read(14)
        #print(cabe)
        while True:
            byte = arquivo.read(1)

            if not byte:
                break
            
            nova_sequencia = buffer + byte

            if nova_sequencia in dicionario:
                buffer = nova_sequencia
            else:
                codigos.append(dicionario[buffer])
                buffer = byte

    if buffer:
        codigos.append(dicionario[buffer])
    
    return len(codigos)

In [94]:
def concatena(arquivos):

    # Nome do arquivo de saída
    arquivo_saida = 'saida.bin'

    # Abrir o arquivo de saída em modo de escrita
    with open(arquivo_saida, 'wb') as arquivo_final:
        # Iterar sobre a lista de arquivos
        for nome_arquivo in arquivos:
            # Abrir cada arquivo de entrada em modo de leitura
            with open(nome_arquivo, 'rb') as arquivo:
                #retira o cabeçalho
                cabe = arquivo.read(14)
                #print(cabe)
                # Ler o conteúdo do arquivo de entrada
                conteudo = arquivo.read()
                #print(conteudo)
                # Escrever o conteúdo no arquivo de saída
                arquivo_final.write(conteudo)
                
            # Adicionar uma quebra de linha entre os arquivos (opcional)
            #arquivo_final.write('\n')
    return arquivo_saida
 

In [95]:
bancos = 40
faces = []
amostras = []

for i in range(bancos):
    pessoa = [f'orl_faces/orl_faces/s{i+1}/1.pgm', f'orl_faces/orl_faces/s{i+1}/2.pgm', f'orl_faces/orl_faces/s{i+1}/3.pgm', f'orl_faces/orl_faces/s{i+1}/4.pgm', f'orl_faces/orl_faces/s{i+1}/5.pgm', f'orl_faces/orl_faces/s{i+1}/6.pgm', f'orl_faces/orl_faces/s{i+1}/7.pgm', f'orl_faces/orl_faces/s{i+1}/8.pgm', f'orl_faces/orl_faces/s{i+1}/9.pgm', f'orl_faces/orl_faces/s{i+1}/10.pgm']
    faces.append(pessoa)

for i in faces:
    indice_aleatorio = random.randint(0, 9)
    amostras.append(i.pop(indice_aleatorio))

print(amostras)
print(f'numero de amostras = {len(amostras)}')

['orl_faces/orl_faces/s1/9.pgm', 'orl_faces/orl_faces/s2/3.pgm', 'orl_faces/orl_faces/s3/9.pgm', 'orl_faces/orl_faces/s4/3.pgm', 'orl_faces/orl_faces/s5/5.pgm', 'orl_faces/orl_faces/s6/8.pgm', 'orl_faces/orl_faces/s7/10.pgm', 'orl_faces/orl_faces/s8/7.pgm', 'orl_faces/orl_faces/s9/5.pgm', 'orl_faces/orl_faces/s10/3.pgm', 'orl_faces/orl_faces/s11/3.pgm', 'orl_faces/orl_faces/s12/7.pgm', 'orl_faces/orl_faces/s13/5.pgm', 'orl_faces/orl_faces/s14/1.pgm', 'orl_faces/orl_faces/s15/7.pgm', 'orl_faces/orl_faces/s16/3.pgm', 'orl_faces/orl_faces/s17/6.pgm', 'orl_faces/orl_faces/s18/4.pgm', 'orl_faces/orl_faces/s19/9.pgm', 'orl_faces/orl_faces/s20/3.pgm', 'orl_faces/orl_faces/s21/7.pgm', 'orl_faces/orl_faces/s22/2.pgm', 'orl_faces/orl_faces/s23/5.pgm', 'orl_faces/orl_faces/s24/8.pgm', 'orl_faces/orl_faces/s25/7.pgm', 'orl_faces/orl_faces/s26/7.pgm', 'orl_faces/orl_faces/s27/4.pgm', 'orl_faces/orl_faces/s28/4.pgm', 'orl_faces/orl_faces/s29/8.pgm', 'orl_faces/orl_faces/s30/10.pgm', 'orl_faces/orl_f

In [96]:
RCs = []
aux2 = []
aux3 = []

for k in range(9,17):

    dicionarios = []

    for i in range(bancos):
        aux = concatena(faces[i])
        dicionarios.append(compressao_LZW(aux,k))

    #print(len(dicionarios))

    for i in amostras:
        for j in dicionarios:
            compressao = rec_pad(i, j)
            file_stats = os.stat(i)
            RC = compressao/file_stats.st_size
            aux2.append(RC)
        #print(len(aux2),aux2)
        aux3.append(aux2.copy())
        aux2.clear()
    RCs.append(aux3.copy())
    aux3.clear()
#print(RCs)
#print(len(RCs))

In [97]:
for k in range(8):
    erro = 0
    acertos = 0
    print(f'k = {k+9}')
    for i in range(len(amostras)):
        #print(f'amostra {i+1}')
        menor = RCs[k][i][i]
        #print(f'prime = {menor}')
        for j in range(len(dicionarios)):
            if(RCs[k][i][j] < menor):
                menor = RCs[k][i][j]
            #print(f'dicionario {j+1}: {RCs[k][i][j]}')
        if(RCs[k][i][i] == menor):
            acertos += 1
        else:
            erro += 1
        #print(f'menor = {menor}')
       
    print(f'acertos = {acertos/(len(amostras))*100}%, erros = {erro/(len(amostras))*100}%')

k = 9
acertos = 20.0%, erros = 80.0%
k = 10
acertos = 12.5%, erros = 87.5%
k = 11
acertos = 30.0%, erros = 70.0%
k = 12
acertos = 35.0%, erros = 65.0%
k = 13
acertos = 62.5%, erros = 37.5%
k = 14
acertos = 67.5%, erros = 32.5%
k = 15
acertos = 92.5%, erros = 7.5%
k = 16
acertos = 92.5%, erros = 7.5%
